In [4]:
import pandas as pd
import numpy as np

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

import matplotlib
matplotlib.rc("font", family = "Malgun Gothic")
matplotlib.rc("axes", unicode_minus = False)

from IPython.display import set_matplotlib_formats
set_matplotlib_formats("retina")

import warnings
warnings.filterwarnings("ignore")
pd.options.display.max_columns = 5000
pd.options.display.max_rows = 1000

In [5]:
train = pd.read_excel("Downloads/데이터분석.xlsx", sheet_name = "상품도달")
train = train.iloc[:,2 :]
print(train.shape)
train.head(2)

(42, 22)


,GMV (취소포함),사용자 유입수,구매전환율,주문건수 (취소포함),장바구니 담기율,1. 1인당 상품상세페이지 뷰,2-1. 리스트페이지 장바구니 클릭 비율,2-2. 리스트페이지 상품 클릭 비율,2-3. 리스트페이지 유입,3-1. 상품상세 메인담기 클릭비율,3-2. 상품상세 유입,4-1. 상품추천영역의 유효클릭 비율,4-2. 상품추천영역의 장바구니 클릭 비율,4-3. 상품추천영역 유입,4-4. 메인페이지 배너클릭 비율,4-5. 기획전 페이지 상품클릭 비율,4-6. 기획전 페이지 장바구니클릭 비율,5. 유입 달성율,6. 전체 품절율,7. 전체 로스율,2-1A. 메인페이지 장바구니 클릭 비율,3-1A. 상품상세페이지 정보영역 클릭비율
0,337388780,218078,0.023487,5122,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,492908190,452930,0.018084,8191,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
imputer = SimpleImputer(strategy='mean')

for i in train.columns:
    if train[i].isnull().sum() !=0:
        train[i] = train[i].fillna(train[i].mean())
        
        
from sklearn.preprocessing import StandardScaler
#
label_name = "GMV (취소포함)"
feature_names = train.columns.difference(["GMV (취소포함)", "주문건수 (취소포함)"])
#
train_x = train[feature_names]
train_y = train[label_name] 
#
train_x_std = StandardScaler().fit_transform(train_x)
train_x_std_final = pd.DataFrame(train_x_std, columns = train_x.columns)

In [7]:
train.corr().head(1).T.sort_values(by = "GMV (취소포함)", ascending = False)

,GMV (취소포함)
GMV (취소포함),1.000000
주문건수 (취소포함),0.967585
3-2. 상품상세 유입,0.892744
장바구니 담기율,0.760821
구매전환율,0.630177
2-1A. 메인페이지 장바구니 클릭 비율,0.572368
2-1. 리스트페이지 장바구니 클릭 비율,0.571352
2-2. 리스트페이지 상품 클릭 비율,0.556364
2-3. 리스트페이지 유입,0.508531
4-3. 상품추천영역 유입,0.302974


In [8]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import log_loss
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error

search=50
hyperparameters_list=[]

for random_search in range(search):
    
    max_depth=np.random.randint(low=50,high=100)
    max_features=np.random.uniform(low=0.1,high=1.0)
    n_estimators = np.random.randint(1000, 3000)
    
    model=RandomForestRegressor(n_jobs=-1,
                                random_state=37, 
                                n_estimators=n_estimators,
                                criterion='mse',
                                max_depth=max_depth,
                                max_features=max_features)
    
    model.fit(train_x_std_final, train_y)
    y_predict = model.predict(train_x_std_final)
    score=mean_absolute_error(train_y, y_predict)
   # score=np.sqrt(score)
    
    hyperparameters={"score":score,"n_estimators":n_estimators,"max_depth":max_depth,"max_features":max_features}
    hyperparameters_list.append(hyperparameters)
    print(f"n_estimators={n_estimators},max_depth={max_depth:.6f},max_features={max_features:.6f},score={score:.1f}")
    
pd.DataFrame(hyperparameters_list).sort_values(by = "score", ascending = True).head(5)

n_estimators=2690,max_depth=95.000000,max_features=0.629426,score=14824083.1
n_estimators=1338,max_depth=96.000000,max_features=0.982519,score=15186417.1
n_estimators=1438,max_depth=72.000000,max_features=0.933996,score=14945156.4
n_estimators=2803,max_depth=53.000000,max_features=0.941637,score=15084187.1
n_estimators=1566,max_depth=91.000000,max_features=0.367908,score=14705848.0
n_estimators=2898,max_depth=58.000000,max_features=0.184325,score=14759996.2
n_estimators=2052,max_depth=68.000000,max_features=0.483246,score=14658699.8
n_estimators=2497,max_depth=64.000000,max_features=0.130277,score=15004201.3
n_estimators=2253,max_depth=53.000000,max_features=0.867536,score=15160202.7
n_estimators=1750,max_depth=73.000000,max_features=0.578819,score=14802094.5
n_estimators=2011,max_depth=62.000000,max_features=0.472369,score=14689021.7
n_estimators=2542,max_depth=62.000000,max_features=0.161672,score=14746958.1
n_estimators=2512,max_depth=97.000000,max_features=0.977568,score=15154441.1

,score,n_estimators,max_depth,max_features
14,1.453420e+07,2250,73,0.267516
49,1.458228e+07,2715,90,0.219504
25,1.458765e+07,1153,97,0.583658
35,1.459370e+07,1595,99,0.254417
31,1.462698e+07,2231,69,0.323717


In [9]:
model = RandomForestRegressor(n_jobs=-1,
                              random_state=37,
                              n_estimators=2250,
                              criterion='mse',
                              max_depth=73,
                              max_features=0.267516)
    
model.fit(train_x_std_final, train_y)
y_predict = model.predict(train_x_std_final)

final = pd.DataFrame({"columns" :train_x_std_final.columns.tolist(), "model_1" : model.feature_importances_}).sort_values(by = "model_1", ascending = False)
final

,columns,model_1
7,3-2. 상품상세 유입,0.233122
17,구매전환율,0.156914
19,장바구니 담기율,0.124352
4,2-3. 리스트페이지 유입,0.107232
1,2-1. 리스트페이지 장바구니 클릭 비율,0.050930
2,2-1A. 메인페이지 장바구니 클릭 비율,0.044075
15,6. 전체 품절율,0.041913
18,사용자 유입수,0.039323
3,2-2. 리스트페이지 상품 클릭 비율,0.030792
5,3-1. 상품상세 메인담기 클릭비율,0.030621


In [10]:
model = RandomForestRegressor(n_jobs=-1,
                              random_state=37,
                              n_estimators=2715,
                              criterion='mse',
                              max_depth=90,
                              max_features=0.219504)
    
model.fit(train_x_std_final, train_y)
y_predict = model.predict(train_x_std_final)

final2 = pd.DataFrame({"columns" :train_x_std_final.columns.tolist(), "model_2" : model.feature_importances_}).sort_values(by = "model_2", ascending = False)
final2

,columns,model_2
7,3-2. 상품상세 유입,0.197026
17,구매전환율,0.145895
19,장바구니 담기율,0.122028
4,2-3. 리스트페이지 유입,0.106753
1,2-1. 리스트페이지 장바구니 클릭 비율,0.052001
2,2-1A. 메인페이지 장바구니 클릭 비율,0.047245
15,6. 전체 품절율,0.045980
18,사용자 유입수,0.042726
5,3-1. 상품상세 메인담기 클릭비율,0.036959
3,2-2. 리스트페이지 상품 클릭 비율,0.035597


In [11]:
pd.merge(final, final2, how = "left", on = "columns")

,columns,model_1,model_2
0,3-2. 상품상세 유입,0.233122,0.197026
1,구매전환율,0.156914,0.145895
2,장바구니 담기율,0.124352,0.122028
3,2-3. 리스트페이지 유입,0.107232,0.106753
4,2-1. 리스트페이지 장바구니 클릭 비율,0.050930,0.052001
5,2-1A. 메인페이지 장바구니 클릭 비율,0.044075,0.047245
6,6. 전체 품절율,0.041913,0.045980
7,사용자 유입수,0.039323,0.042726
8,2-2. 리스트페이지 상품 클릭 비율,0.030792,0.035597
9,3-1. 상품상세 메인담기 클릭비율,0.030621,0.036959
